In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cooler
import lavaburst

/home/dmitry/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import TADcalling
from TADcalling.CallerClasses import *

In [3]:
hc = HiCsegCaller(['S2'], ['./data/S2.20000.cool'], 'cool', assembly='dm3', resolution=20000, balance=True, chr='chr2L')

In [10]:
hc.call(distr_model="P", tune=False)

{'S2': {'B': 375	712	1
  712	1151	1, 'P': 375	712	1
  712	1069	1
  1069	1077	1
  1077	1151	1}}

In [7]:
IS = InsulationCaller(['S2'], ['./data/S2.20000.cool'], 'cool', assembly='dm3', resolution=20000, balance=True, chr='chr2L')

In [8]:
IS.call(2, 0.5)

{'S2': {(2, 0.5): 785	1008	1
  1007	1011	1
  1010	1070	1
  1076	1137	1
  1136	1140	1}}

In [6]:
DI = DirectionalityCaller(['S2'], ['./data/S2.20000.cool'], 'cool', assembly='dm3', resolution=20000, balance=True, chr='chr2L')

In [9]:
DI.call(2, 0.5)

{'S2': {(2, 0.5): 17	48	1
  49	130	1
  131	229	1
  230	400	1
  401	662	1
  663	782	1
  785	966	1
  967	988	1
  989	1008	1
  1010	1021	1
  1022	1050	1
  1051	1070	1
  1076	1137	1
  1139	1149	1}}

In [3]:
lc = LavaburstCaller(['S2'], ['./data/S2.20000.cool'], 'cool', assembly='dm3', resolution=20000, balance=True, chr='chr2L')

DEVELOPMENT OF MrTADFinder

In [6]:
c = cooler.Cooler('./data/S2.20000.cool')

In [29]:
sparse = c.matrix(balance=True, as_pixels=True).fetch('chr2L')

In [58]:
sparse.loc[:, "bin1_id":"bin2_id"] += 1

In [59]:
sparse

,bin1_id,bin2_id,count,balanced
0,1,1,2859,0.892852
1,1,2,382,0.117808
2,1,3,315,0.076218
3,1,4,251,0.051895
4,1,5,109,0.026401
5,1,6,61,0.018097
6,1,7,53,0.019344
7,1,8,39,0.013315
8,1,9,16,0.005216
9,1,10,34,0.009732


In [60]:
sparse.loc[:, 'bin1_id':'count'].to_csv('test_sparse.txt', header=False, index=False, sep='\t')

In [61]:
max_bin = sparse.loc[:, 'bin1_id':'bin2_id'].max().max()

In [77]:
with open("genome_bin.txt", 'w') as outfile:
    outfile.write("1\tchr1\t0\t{}".format(max_bin - 1))

In [79]:
max_bin

1151

In [80]:
with open('chr_bin.txt', 'w') as outfile:
    for i in range(max_bin):
        outfile.write("0\t{}\t{}\n".format(i * 20000 + 1, (i + 1) * 20000))

In [78]:
subprocess.run("echo sharlotka | sudo julia ../MrTADFinder/run_MrTADFinder.jl test_sparse.txt genome_bin.txt chr_bin.txt res=1 1 test_mrtadfinder.txt", shell=True)

CompletedProcess(args='echo sharlotka | sudo julia ../MrTADFinder/run_MrTADFinder.jl test_sparse.txt genome_bin.txt chr_bin.txt res=1 1 test_mrtadfinder.txt', returncode=1)

In [4]:
lc.call(0.9)

{'S2': {0.9: 0	6	1
  6	15	1
  21	27	1
  27	83	1
  84	108	1
  108	112	1
  112	131	1
  132	137	1
  137	143	1
  143	149	1
  152	173	1
  173	183	1
  183	222	1
  223	230	1
  231	241	1
  241	248	1
  248	253	1
  253	264	1
  266	277	1
  277	298	1
  299	304	1
  304	324	1
  324	333	1
  333	348	1
  348	352	1
  352	361	1
  361	371	1
  371	375	1
  375	391	1
  391	400	1
  405	412	1
  412	417	1
  417	421	1
  421	427	1
  427	449	1
  449	473	1
  473	485	1
  485	495	1
  496	511	1
  511	521	1
  521	525	1
  525	537	1
  537	549	1
  549	557	1
  557	600	1
  603	622	1
  622	627	1
  627	650	1
  650	660	1
  664	669	1
  669	677	1
  677	682	1
  682	690	1
  691	695	1
  695	714	1
  714	753	1
  753	783	1
  786	813	1
  815	826	1
  826	837	1
  837	934	1
  934	949	1
  949	958	1
  958	967	1
  971	979	1
  979	989	1
  990	1004	1
  1004	1009	1
  1011	1022	1
  1023	1039	1
  1039	1046	1
  1046	1051	1
  1052	1057	1
  1057	1062	1
  1062	1071	1
  1077	1126	1
  1126	1151	1}}

In [5]:
GenomicRanges

TADcalling.DataClasses.GenomicRanges

In [6]:
lc.convert_files("txt.gz")

In [18]:
lc._metadata

{'assembly': 'dm3',
 'balance': False,
 'chr': 'chr2L',
 'data_formats': ['cool', 'txt.gz', 'txt.gz'],
 'files_cool': ['./data/S2.20000.cool'],
 'files_txt.gz': ['./data/S2.20000.chr2L.txt.gz'],
 'labels': ['S2'],
 'resolution': 1000,
 'size': 0}

In [7]:
acpp = ArmatusCaller(['S2'], ['./data/S2.20000.cool'], 'cool', assembly='dm3', resolution=20000, balance=True, chr='chr2L')

In [8]:
acpp.call(-0.3)

{'S2': {-0.3: array([[   6,   14],
         [  26,  400],
         [ 483,  489],
         [ 536,  542],
         [ 598,  645],
         [ 658,  662],
         [ 682,  716],
         [ 816,  833],
         [ 900,  907],
         [ 937,  941],
         [ 968, 1050],
         [1116, 1127]])}}

In [10]:
a = "abc" \
    "def" \
    "ghi"
print(a)

abcdefghi
